## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [9]:
import pandas as pd


df_movies = pd.read_csv('../../datasets/tmdb_5000_movies.csv', parse_dates=['release_date'])
df_credit = pd.read_csv('../../datasets/tmdb_5000_credits.csv')
df_all = pd.merge(df_movies, df_credit, left_on='id', right_on='movie_id', suffixes=('_x', None))
df_all = df_all[df_movies.status == 'Released'].copy()
df_all_f = df_all.dropna(subset=['overview', 'genres', 'keywords']).copy()
print('Количество фильмов после фильтрации:', df_all_f.shape[0])

Количество фильмов после фильтрации: 4792


## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import ast


df_all_f['combi'] = df_all_f.keywords.apply(lambda x: " ".join([str['name'] for str in ast.literal_eval(x)]))
df_all_f['combi'] = df_all_f.overview + ' ' + df_all_f.combi
df_all_f['date'] = df_all_f['release_date'].dt.year
df_all_f['genres1'] = df_all_f.genres.apply(lambda x: " ".join([str['name'] for str in ast.literal_eval(x)]))
vec = TfidfVectorizer(stop_words='english', max_features=10000)

matrix = vec.fit_transform(df_all_f.combi)
matrix.shape

(4792, 10000)

## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [11]:
from sklearn.metrics.pairwise import cosine_similarity


cosine_sim = cosine_similarity(matrix, matrix)
df_distance = pd.DataFrame(cosine_sim, index=df_all_f.id, columns=df_all_f.id)

In [12]:
df_distance.to_csv('../assets/distance.csv')
df_all_f.to_csv('../assets/movies.csv')